In [29]:
#JJ Rolling Pivots

import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [30]:
bnf_1hr = pd.read_csv("../data/midcp_1hr_tv.csv")
bnf_1hr["datetime"] = pd.to_datetime(bnf_1hr["time"])
bnf_1hr = bnf_1hr.drop(columns=["time"])
bnf_1hr = bnf_1hr[(bnf_1hr["datetime"].dt.year >= 2022)]
bnf_1hr.head()

,open,high,low,close,datetime
0,7690.1000,7744.3500,7684.7000,7729.9500,2022-01-10 09:15:00+05:30
1,7730.3000,7739.4500,7721.5500,7738.2500,2022-01-10 10:15:00+05:30
2,7738.6000,7739.3000,7707.6500,7717.4500,2022-01-10 11:15:00+05:30
3,7717.4000,7730.7000,7716.2000,7727.2000,2022-01-10 12:15:00+05:30
4,7726.9000,7731.4000,7719.5000,7727.3000,2022-01-10 13:15:00+05:30


In [31]:
# spot_data = pd.read_csv('../data/midcp_min.csv')
# spot_data['datetime'] = pd.to_datetime(spot_data['datetime'])
# spot_data = pl.DataFrame(spot_data)
# spot_data = spot_data.with_columns([pl.col('datetime').alias('index')])

# def resample(
#     data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
# ) -> pl.DataFrame:
#     return (
#         data.set_sorted("datetime")
#         .group_by_dynamic(
#             index_column="datetime",
#             every=timeframe,
#             period=timeframe,
#             truncate=True,
#             offset=offset,
#         )
#         .agg(
#             [
#                 pl.col("open").first().alias("open"),
#                 pl.col("high").max().alias("high"),
#                 pl.col("low").min().alias("low"),
#                 pl.col("close").last().alias("close"),
#                 pl.col("volume").sum().alias("volume"),
#             ]
#         )
#     )

# bnf_resampled = resample(spot_data, '60m', pd.Timedelta(minutes=15))

# bnf_df = bnf_resampled.to_pandas()
# bnf_1hr = bnf_df

In [32]:
bnf_1hr['close'] = pd.to_numeric(bnf_1hr['close'], errors='coerce')
bnf_1hr['MA'] = bnf_1hr['close'].rolling(window=36).mean()

In [33]:
PORTFOLIO = 10_00_000
INDEX_LEVERAGE = 3
SLIPPAGE_FACTOR = 0.0004

In [34]:
def generate_pivots2(df):

    df["Bullish Pivot"] = 0
    df["Bearish Pivot"] = 0
    df["MA"] = df["close"].rolling(window=18).mean()

    # Generate signals using boolean masking
    bull_pivot_mask = (df["close"] >= df["close"].shift(1)) & (
        df["close"].shift(2) >= df["close"].shift(1)
    )
    bear_pivot_mask = (df["close"] <= df["close"].shift(1)) & (
        df["close"].shift(2) <= df["close"].shift(1)
    )

    df.loc[bull_pivot_mask, "Bullish Pivot"] = 1
    df.loc[bear_pivot_mask, "Bearish Pivot"] = 1

    return df


bnf_1hr = generate_pivots2(bnf_1hr)
bnf_1hr.head()

,open,high,low,close,datetime,MA,Bullish Pivot,Bearish Pivot
0,7690.1000,7744.3500,7684.7000,7729.9500,2022-01-10 09:15:00+05:30,NaN,0,0
1,7730.3000,7739.4500,7721.5500,7738.2500,2022-01-10 10:15:00+05:30,NaN,0,0
2,7738.6000,7739.3000,7707.6500,7717.4500,2022-01-10 11:15:00+05:30,NaN,0,1
3,7717.4000,7730.7000,7716.2000,7727.2000,2022-01-10 12:15:00+05:30,NaN,1,0
4,7726.9000,7731.4000,7719.5000,7727.3000,2022-01-10 13:15:00+05:30,NaN,0,0


In [35]:
temp_df = bnf_1hr


def execute_trades(df):
    tradebook = []

    in_trade = False
    long_trade_active = False
    short_trade_active = False

    remark = ""

    signal_entry_price = 1_00_000
    signal_initial_sl = 0

    points = 0

    long_points = 0
    short_points = 0

    for i in range(0, len(df)):

        points = 0

        previous_candle_open = df.iloc[i - 1]["open"]
        previous_candle_high = df.iloc[i - 1]["high"]
        previous_candle_low = df.iloc[i - 1]["low"]
        previous_candle_close = df.iloc[i - 1]["close"]
        previous_candle_time = df.iloc[i - 1]["datetime"]
        previous_candle = df.iloc[i-1]

        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]
        current_candle_time = df.iloc[i]["datetime"]
        current_candle = df.iloc[i]

        print(
            current_candle_time,
            current_candle_open,
            current_candle_high,
            current_candle_low,
            current_candle_close,
        )

        if not in_trade:

            if (df.iloc[i - 1]["Bullish Pivot"] == 1) & (
                df.iloc[i - 1]["Bearish Pivot"] == 1
            ):
                if current_candle_close < df.iloc[i]['MA']:
                    df.at[i - 1, "Bullish Pivot"] = 0
                elif current_candle_close >= df.iloc[i]['MA']:
                    df.at[i - 1, "Bearish Pivot"] = 0

            if df.iloc[i - 1]["Bullish Pivot"] == 1:
                # Long Side
                signal_time = previous_candle_time

                if current_candle_close >= previous_candle_close and (current_candle_close >= df.iloc[i]['MA']):
                    # if current_candle_open > previous_candle_high:
                    #     # Check Entry Skip
                    #     print("Long Entry Skipped", current_candle_time)
                    #     in_trade = False
                    #     signal_entry_price = 1_00_000
                    #     signal_initial_sl = 0
                    #     continue
                    # else:
                        # Entry Triggered
                    print("Long Entry Triggered", current_candle_time)
                    in_trade = True
                    long_trade_active = True
                    entry_time = current_candle_time
                    entry_price = current_candle_close
                    initial_sl = df.iloc[i - 2]["low"]
                    qty = PORTFOLIO * INDEX_LEVERAGE / entry_price
                        
            if df.iloc[i - 1]["Bearish Pivot"] == 1:
                # Short Side
                signal_time = previous_candle_time

                if current_candle_close <= previous_candle_close and (current_candle_close < df.iloc[i]['MA']):
                    # if current_candle_open < previous_candle_low:
                    #     # Check Entry Skip
                    #     print("Short Entry Skipped", current_candle_time)
                    #     in_trade = False
                    #     signal_entry_price = 1_00_000
                    #     signal_initial_sl = 0
                    #     continue
                    # else:
                        # Entry Triggered
                    print("Short Entry Triggered", current_candle_time)
                    in_trade = True
                    short_trade_active = True
                    entry_time = current_candle_time
                    entry_price = current_candle_close
                    initial_sl = df.iloc[i - 2]["high"]
                    qty = PORTFOLIO * INDEX_LEVERAGE / entry_price

        if in_trade:
            if long_trade_active:
                if current_candle_open < initial_sl:
                    # Gap Outside Initial SL
                    in_trade = False
                    long_trade_active = False
                    exit_price = current_candle_open
                    exit_time = current_candle_time
                    long_points = exit_price - entry_price
                    remark = "Gap Outside Initial SL"
                    print(remark, current_candle_time, exit_price)

                elif (df.iloc[i - 1]["Bearish Pivot"] == 1) and (current_candle_close <= previous_candle_close) and (not(current_candle_open < previous_candle_low)):
                # if (current_candle_low < previous_candle_low) and (not(current_candle_open < previous_candle_low)): # Ensuring that there was no Gap for entry
                    # Reversal Trade
                    exit_price = current_candle_close
                    exit_time = current_candle_time
                    long_points = exit_price - entry_price
                    remark = "Reversed"
                    print(remark, current_candle_time, exit_price)
                    trade = {
                        "Trade Type": "LONG",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": long_points,
                        "Points w CS": long_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * long_points,
                        "PnL w CS": qty
                        * (
                            long_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * long_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (
                            long_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        )
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)
    
                    long_points = 0

                    long_trade_active = False
                    short_trade_active = True
                    in_trade = True

                    entry_price = previous_candle_close
                    initial_sl = df.iloc[i - 2]["high"]
                    signal_time = previous_candle_time
                    entry_time = current_candle_time
                    print(signal_time, entry_time, entry_price, initial_sl)
                    qty = PORTFOLIO * INDEX_LEVERAGE / entry_price
                    # continue

                    if current_candle_close >= initial_sl:
                        if not (current_candle_open > initial_sl): # Ensuring that there was no Gap for entry
                            # Initial SL Hit
                            # print(current_candle)
                            in_trade = False
                            long_trade_active = False
                            exit_price = current_candle_close
                            exit_time = current_candle_time
                            short_points = entry_price - exit_price
                            remark = "Initial SL Hit"
                            print(remark, current_candle_time, exit_price)

                            trade = {
                                "Trade Type": "SHORT",
                                "Signal Generated At": signal_time,
                                "Entry Time": entry_time,
                                "Entry Price": entry_price,
                                "Initial SL": initial_sl,
                                "Exit Time": exit_time,
                                "Exit Price": exit_price,
                                "Remarks": remark,
                                "Points Captured": short_points,
                                "Points w CS": short_points
                                - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                                "Qty": qty,
                                "Leverage": INDEX_LEVERAGE,
                                "PnL": qty * short_points,
                                "PnL w CS": qty
                                * (
                                    short_points
                                    - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                                ),
                                "ROI%": qty * short_points * 100 / PORTFOLIO,
                                "ROI% w CS": qty
                                * (
                                    short_points
                                    - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                                )
                                * 100
                                / PORTFOLIO,
                                "Trade Year": entry_time.year,
                            }
                            tradebook.append(trade)
        
                            short_points = 0
                            short_trade_active = False
                            

                elif current_candle_close <= initial_sl:
                    # Initial SL Hit
                    in_trade = False
                    long_trade_active = False
                    exit_price = current_candle_close
                    exit_time = current_candle_time
                    long_points = exit_price - entry_price
                    remark = "Initial SL Hit"
                    print(remark, current_candle_time, exit_price)

                if long_points:
                    trade = {
                        "Trade Type": "LONG",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": long_points,
                        "Points w CS": long_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * long_points,
                        "PnL w CS": qty
                        * (
                            long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * long_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price)))
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    long_points = 0
                    long_trade_active = False

            if short_trade_active:
                if current_candle_open > initial_sl:
                    # Gap Outside Initial SL
                    in_trade = False
                    short_trade_active = False
                    exit_price = current_candle_open
                    exit_time = current_candle_time
                    short_points = entry_price - exit_price
                    remark = "Gap Outside Initial SL"
                    print(remark, current_candle_time, exit_price)

                elif (df.iloc[i - 1]["Bullish Pivot"] == 1) and (current_candle_high > previous_candle_high) and (not(current_candle_open > previous_candle_high)):
                    # if (current_candle_high > previous_candle_high) and (not(current_candle_open > previous_candle_high)): # Ensuring that there was no Gap for entry
                    # Reversal Tradce
                    exit_price = current_candle_close
                    exit_time = current_candle_time
                    short_points = entry_price - exit_price
                    remark = "Reversed"
                    print(remark, current_candle_time, exit_price)
                    trade = {
                        "Trade Type": "SHORT",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": short_points,
                        "Points w CS": short_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * short_points,
                        "PnL w CS": qty
                        * (
                            short_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * short_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (
                            short_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        )
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    short_points = 0

                    long_trade_active = True
                    short_trade_active = False
                    in_trade = True

                    entry_price = current_candle_close
                    initial_sl = df.iloc[i - 2]["low"]
                    signal_time = previous_candle_time
                    entry_time = current_candle_time
                    qty = PORTFOLIO * INDEX_LEVERAGE / entry_price
                    print(signal_time, entry_time, entry_price, initial_sl)
                    # continue

                    if current_candle_close <= initial_sl:
                        if not (current_candle_open < initial_sl): # Ensuring that there was no Gap for entry
                            # Initial SL Hit
                            # print(current_candle)
                            in_trade = False
                            short_trade_active = False
                            exit_price = current_candle_close
                            exit_time = current_candle_time
                            long_points = exit_price - entry_price
                            remark = "Initial SL Hit"
                            print(remark, current_candle_time, exit_price)

                            trade = {
                                "Trade Type": "LONG",
                                "Signal Generated At": signal_time,
                                "Entry Time": entry_time,
                                "Entry Price": entry_price,
                                "Initial SL": initial_sl,
                                "Exit Time": exit_time,
                                "Exit Price": exit_price,
                                "Remarks": remark,
                                "Points Captured": long_points,
                                "Points w CS": long_points
                                - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                                "Qty": qty,
                                "Leverage": INDEX_LEVERAGE,
                                "PnL": qty * long_points,
                                "PnL w CS": qty
                                * (
                                    long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                                ),
                                "ROI%": qty * long_points * 100 / PORTFOLIO,
                                "ROI% w CS": qty
                                * (long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price)))
                                * 100
                                / PORTFOLIO,
                                "Trade Year": entry_time.year,
                            }
                            tradebook.append(trade)
        
                            long_points = 0
                            long_trade_active = False

                elif current_candle_close >= initial_sl:
                    # Initial SL Hit
                    in_trade = False
                    short_trade_active = False
                    exit_price = current_candle_close
                    exit_time = current_candle_time
                    short_points = entry_price - exit_price
                    remark = "Initial SL Hit"
                    print(remark, current_candle_time, exit_price)

                if short_points:
                    trade = {
                        "Trade Type": "SHORT",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": short_points,
                        "Points w CS": short_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * short_points,
                        "PnL w CS": qty
                        * (
                            short_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * short_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (
                            short_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        )
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    short_points = 0
                    short_trade_active = False

    trade_book_df = pd.DataFrame(tradebook)
    return trade_book_df


tb = execute_trades(bnf_1hr)

2022-01-10 09:15:00+05:30 7690.1 7744.35 7684.7 7729.95
2022-01-10 10:15:00+05:30 7730.3 7739.45 7721.55 7738.25
2022-01-10 11:15:00+05:30 7738.6 7739.3 7707.65 7717.45
2022-01-10 12:15:00+05:30 7717.4 7730.7 7716.2 7727.2
2022-01-10 13:15:00+05:30 7726.9 7731.4 7719.5 7727.3
2022-01-10 14:15:00+05:30 7727.6 7742.5 7721.85 7737.1
2022-01-10 15:15:00+05:30 7736.65 7739.45 7729.5 7736.8
2022-01-11 09:15:00+05:30 7736.9 7783.25 7731.05 7760.95
2022-01-11 10:15:00+05:30 7760.85 7773.25 7744.6 7767.15
2022-01-11 11:15:00+05:30 7767.4 7784.85 7763.65 7771.15
2022-01-11 12:15:00+05:30 7770.85 7794.8 7767.7 7788.9
2022-01-11 13:15:00+05:30 7789.15 7790.95 7773.8 7787.45
2022-01-11 14:15:00+05:30 7786.05 7790.05 7748.55 7775.8
2022-01-11 15:15:00+05:30 7776.45 7781.6 7771.25 7776.2
2022-01-12 09:15:00+05:30 7812.85 7837.85 7785.35 7825.9
2022-01-12 10:15:00+05:30 7826.0 7831.0 7815.65 7820.0
2022-01-12 11:15:00+05:30 7820.1 7831.65 7814.85 7830.15
2022-01-12 12:15:00+05:30 7830.4 7858.95 7829.6

In [1]:
# tb

In [2]:
# tb["ROI% w CS"].sum()

In [38]:
stats_df8 = pd.DataFrame(
    index=range(2022, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb
# Iterate over each year
for year in range(2022, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w CS"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w CS"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w CS"] > 0]["ROI% w CS"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w CS"] < 0]["ROI% w CS"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w CS"].cumsum() - year_trades["ROI% w CS"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w CS"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w CS"] > 0][
    "ROI% w CS"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w CS"] < 0][
    "ROI% w CS"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w CS"].cumsum()
    - combined_df_sorted["ROI% w CS"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2022,134.8880,267,40.8240,3.6390,-1.6568,-29.4659,4.5778
2023,94.3899,273,40.2930,2.3918,-1.0350,-15.7036,6.0107
2024,37.8896,119,37.8151,3.4661,-1.5957,-25.2740,1.4992
Overall,267.1676,659.0000,40.0607,3.0899,-1.3888,-29.4659,9.0670


# MIDCP Rolling Pivots with 36MA Buy/Sell Distinction 0.04% Slippage Pivot of 3 candles CLOSING BASIS

In [3]:
# tb.tail()

In [46]:
tb.to_csv(f'MIDCP Rolling Pivots {INDEX_LEVERAGE}x Lev.csv')